
# Logging in Python: Observability and Design

In production-grade software, your code is rarely run under your direct supervision. Instead, it operates within remote systems, embedded services, or cloud environments—often out of sight and in constant motion. In such contexts,

When an error happens, you need to have a log that you or a software tester can capture to troubleshoot. 

This means using the standard output (ie. Print command) will not be useful for you or your users during runtime. So far you have been logging detailed information about internal application state, warnings, and errors that happen during program execution. 

> "If code is what tells the system what to do, then logging is how the system tells *you* what it did."

## Why Logging Matters

Consider this:
- You're not watching your program run.
- Failures might only be discovered hours or days after they occur.
- You need post-mortem forensic capabilities.

Thus, logging isn't a luxury; it is your first line of **introspection, debugging, and accountability**.

### Standard Output ≠ Logging

While `print()` statements may help during early development, they fall short in reliability and configurability:
- Cannot be easily turned off/on.
- Cannot be redirected, filtered, or structured.
- Do not convey severity or source.

Python comes with a built-in `logging` module that provides a basic yet powerful logging system. 

## Setting Up a Logger 
```python
logger = logging.getLogger(__name__)

debuglog = logging.getLogger("my_logger")
```
\_\_name\_\_" will be the named logger 

    More on this when whe get to flags 

In [ ]:
import logging

logging.basicConfig() # Basic Config is output to the terminal at warning and above

# Name the logger
defautLogger = logging.getLogger("defautLogger")
defautLogger.info("this is a basic info log")


## Keywords
Each log message is associated with a **severity level**, providing a mechanism to filter and prioritize information. These are:

| Level      | Description |
|------------|-------------|
| `CRITICAL` / `FATAL` | System is unusable. Cannot continue. |
| `ERROR`    | Major failure. An operation failed. |
| `WARNING`  | Anomalies that are recoverable, but notable. |
| `INFO`     | Confirmation that things are working as expected. |
| `DEBUG`    | Diagnostic output useful during development. |
| `NOTSET`   | Captures everything (used for internal fallback). |

In [ ]:
#there are 2 ways to output logs 

defautLogger.log(logging.CRITICAL, "OMG THE WORLD IS ENDING!!!!")

defautLogger.critical("there is no more disk space")
defautLogger.error("file not found creating new file")
defautLogger.warning("disk space is low")
defautLogger.info("user logged in with correct password")

#they do the same thing

CRITICAL:systemConfiguredLogger:OMG THE WORLD IS ENDING!!!!
CRITICAL:systemConfiguredLogger:there is no more disk space
ERROR:systemConfiguredLogger:file not found creating new file


Notice how info is not displayed this is because when we setup up logging above we set it up using the default basic config 

``` python
logging.basicConfig() # Basic Config is output to the terminal at warning and above
```

### Custom logging config

In [ ]:
import logging

logging.basicConfig(level=logging.DEBUG, format='%(levelname)s: %(message)s')

# Name the logger 
debugLogger = logging.getLogger("my_logger")

debugLogger.log(logging.CRITICAL, "OMG THE WORLD IS ENDING!!!!")

debugLogger.critical("there is no more disk space")
debugLogger.error("file not found creating new file")
debugLogger.warning("disk space is low")
debugLogger.info("user logged in with correct password")


## Wait They look the same

Because Jupyter notebooks persist state, if you configure logging once, it will retain the configuration. 

There are two ways to reconfigure cleanly:

```python
import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
```

Or simply use: for versions of python 3.8 and above

```python
logging.basicConfig(..., force=True)
```

In [ ]:

logging.basicConfig(level=logging.INFO, format='%(name)s %(levelname)s: %(message)s',force=True)

infoLogger = logging.getLogger("my_logger")

infoLogger.critical("There is no more disk space")
infoLogger.error("File not found creating new file")
infoLogger.warning("Disk space is low")
infoLogger.info("User logged in with correct password")

my_logger CRITICAL: There is no more disk space
my_logger ERROR: File not found creating new file
my_logger WARNING: Disk space is low
my_logger INFO: User logged in with correct password


### Logging Handlers
Handlers determine **where log messages go**.

### Built-in Handler Types

| Handler              | Description |
|----------------------|-------------|
| `StreamHandler`      | Sends logs to streams (stdout, stderr). Used by default. |
| `FileHandler`        | Writes logs to disk files. |
| `NullHandler`        | Ignores all logging messages. Useful for library authors. |
| `TimedRotatingFileHandler` | Rotates logs at time intervals (e.g., daily). |
| `RotatingFileHandler`      | Rotates logs when file size exceeds a limit. |


## FileHandler
File Handler performs output to a log

In [ ]:
logger = logging.getLogger("file_logger")
logger.setLevel(logging.INFO)

file_handler = logging.FileHandler("example_log.log")
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))

logger.addHandler(file_handler)

logger.info("Logging to a file!")
logger.warning("Watch out! Disk space might be low.")

#what happened why did we see the logged output in the terminal?

file_logger INFO: Logging to a file!
file_logger WARNING: Watch out! Disk space might be low.


2025-04-13 05:54:52,477 - INFO - Logging to a file!
2025-04-13 05:54:52,478 - WARNING - Watch out! Disk space might be low.
2025-04-13 06:00:01,630 - INFO - Logging to a file!
2025-04-13 06:00:01,630 - file_logger - INFO - Logging to a file!
2025-04-13 06:00:01,631 - WARNING - Watch out! Disk space might be low.
2025-04-13 06:00:01,631 - file_logger - WARNING - Watch out! Disk space might be low.
2025-04-13 06:03:32,819 - INFO - Logging to a file!
2025-04-13 06:03:32,819 - file_logger - INFO - Logging to a file!
2025-04-13 06:03:32,819 - file_logger - INFO - Logging to a file!
2025-04-13 06:03:32,820 - WARNING - Watch out! Disk space might be low.
2025-04-13 06:03:32,820 - file_logger - WARNING - Watch out! Disk space might be low.
2025-04-13 06:03:32,820 - file_logger - WARNING - Watch out! Disk space might be low.


## Time Rotating logging 
(my personal favorite)

In [13]:
# FileHandler - timedRotating

import logging.handlers
from datetime import timedelta, datetime

logger = logging.getLogger("timeRotatingLogger")

logger.setLevel(logging.INFO) # set the logging level to INFO

# create a file handler that logs to a file
logger.addHandler(
    logging.handlers.TimedRotatingFileHandler(
        filename="archived_log.log", #file name
        when="D", #rolls the log every day
        backupCount=3, # only keep 3 days worth of file backups
    )
)


## Gu Loves his freezegun
    Nothing is better then a freeze-gun 
The freezegun package is used to trick Python into thinking that time is passing at an increased pace and is very useful in testing

In [ ]:
!pip install freezegun

In [14]:

from datetime import timedelta, datetime
import time
import freezegun

def main():
    with freezegun.freeze_time() as frozen:
        for _ in range(10):
            frozen.tick(timedelta(hours=24))
            time.sleep(0.1)
            logger.info(f"all your base are belong to us at {datetime.now()}")

if __name__ == "__main__":
    main()

timeRotatingLogger INFO: all your base are belong to us at 2025-04-14 13:06:54.656594
timeRotatingLogger INFO: all your base are belong to us at 2025-04-15 13:06:54.656594
timeRotatingLogger INFO: all your base are belong to us at 2025-04-16 13:06:54.656594
timeRotatingLogger INFO: all your base are belong to us at 2025-04-17 13:06:54.656594
timeRotatingLogger INFO: all your base are belong to us at 2025-04-18 13:06:54.656594
timeRotatingLogger INFO: all your base are belong to us at 2025-04-19 13:06:54.656594
timeRotatingLogger INFO: all your base are belong to us at 2025-04-20 13:06:54.656594
timeRotatingLogger INFO: all your base are belong to us at 2025-04-21 13:06:54.656594
timeRotatingLogger INFO: all your base are belong to us at 2025-04-22 13:06:54.656594
timeRotatingLogger INFO: all your base are belong to us at 2025-04-23 13:06:54.656594


In [15]:
!cat archived_log.log*
#windows
#!type archived_log.log*


all your base are belong to us at 2025-04-23 13:06:54.656594
all your base are belong to us at 2025-04-20 13:06:54.656594
all your base are belong to us at 2025-04-21 13:06:54.656594
all your base are belong to us at 2025-04-22 13:06:54.656594


In [ ]:
!rm archived_log.log*
#Windows
#!del archived_log.log*

## Formatting  

Notice that the above log does not have the "INFO" tag and that is because it was not configured. 


Lets correct this by using the formater 

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO) #log Level
formatter = logging.Formatter(
    fmt=(
        "%(asctime)s | %(levelname)s | "
        "%(message)s"
    )
)
handler = logging.handlers.TimedRotatingFileHandler(
    filename="formatedLog.log",
    when="D",
    backupCount=3,
)
handler.setFormatter(formatter)
logger.addHandler(handler)

*Formater is customizable see below:*

**fmt** - This is a string formatting pattern for the output message. 

**datefmt** - This is a date formatting pattern for representing the message timestamp. 

**style** - This specifies the string formatting style used for the fmt argument. It can be either '%' (percent formatting), '{' (str.format() formatting), or '$' (string.Template formatting). It defaults to '%'.

**validate** - This specifies whether to validate the fmt formatting argument against the style argument. It defaults to True.

In [ ]:
from datetime import timedelta, datetime
import time
import logging
import freezegun
def main():
    with freezegun.freeze_time() as frozen:
        for _ in range(5):
            frozen.tick(timedelta(hours=24))
            time.sleep(0.1)
            for _ in range(100):
                logger.warning(f"all your base are belong to us")
                logger.critical(f"all your base are belong to us")
if __name__ == "__main__":
    main()

In [ ]:
!cat formatedLog.log

### Costomized Logging
in your adventures you may find that some libraries log differently then what you desire. 






In [ ]:
# silance the log
logger.disabled = False
#Overriding handlers
logger.handlers.clear()
#Changing logger verbosity
logger.setLevel(logging.WARNING)

### Fun with ~flags~ logs 

Periods are an import part of logs. A period will build a hierarchy of loggers.

for example  logger.lib.util

The system will create/search for logger registered under logger.lib.util

Then will create/search for logger registered as logger.lib then logger. 
parent
note if logger.util or logger are not found logger.lib.util will become the parent . 

Best practice is to define log handlers only at the top-level root logger. Otherwise, it will be hard to track all propagation rules and make sure that every message is logged only once


```python
logging.getLogger("my_app")
logging.getLogger("my_app.utils")
logging.getLogger("my_app.utils.db")
```

These logger names form a **tree-like structure**:

```
ROOT
   │
   └── my_app
         │
         └── utils
               │
               └── db
   └── log
```

a log message travels **upward through the hierarchy**, from the child to the parent, until it finds a logger with a handler.

Here's how log messages propagate:

```
[my_app.utils.db] ---log---> [my_app.utils] ---log---> [my_app] ---log---> [root logger]
```



In [ ]:
import logging
parentLog = logging.getLogger("my_logger")
parentLog.setLevel(logging.INFO)
#parentLog.setLevel(logging.CRITICAL)
childLog = logging.getLogger("../log/child_logger")

childLog.setLevel(logging.CRITICAL)
#childLog.setLevel(logging.INFO)
formatter = logging.Formatter(
    fmt=(
        "%(asctime)s | %(name)s | %(levelname)s | "
        "%(message)s"
    )
)
handler = logging.handlers.TimedRotatingFileHandler(
    filename="parent.log",
    when="D",
    backupCount=3,
)
handler2 = logging.handlers.TimedRotatingFileHandler(
    filename="child.log",
    when="D",
    backupCount=3,
)
handler.setFormatter(formatter)
handler2.setFormatter(formatter)
parentLog.addHandler(handler)
childLog.addHandler(handler2)

In [ ]:
parentLog.critical(f"all your base are belong to us")
parentLog.info(f"hello")

In [ ]:
childLog.critical(f"child me")
childLog.info(f"hello")

In [ ]:
!cat child.log
!cat parent.log

In [ ]:
!rm child.log
!rm parent.log